In [2]:
#터미널에서 설치
#pip install azure-cognitiveservices-vision-customvision
#pip install pylabel


In [2]:
#Import Azure cognitive services libraries 
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials

import os, zipfile
from pathlib import PurePath
from os.path import exists
from decimal import *

from pylabel import importer

In [ ]:
ENDPOINT = "Endpoint"
training_key = "Training Key"
prediction_key = "Training Key와 동일"
prediction_resource_id = "구독 ID"

In [4]:
#Initialize objects used by Azure Congitive vision
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(ENDPOINT, prediction_credentials)

#Create a new project
publish_iteration_name = "Classification_Model"
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "Classification" and domain.name == "General")
project = trainer.create_project("Team5", domain_id=obj_detection_domain.id)
#If you browse to https://www.customvision.ai/ you should see a new project called "PyLabel Sample Dataset"

In [6]:
import os
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd

class MultiLabelImageDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.labels_frame = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        self.classes = ['male', 'female', 'old', 'adult', 'young']

    def __len__(self):
        return len(self.labels_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.labels_frame.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')
        
        labels = self.labels_frame.iloc[idx, 1:].values.astype('float')
        
        if self.transform:
            image = self.transform(image)

        return image, labels

# 데이터 변환 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 데이터셋 생성
dataset = MultiLabelImageDataset(csv_file='dataset/gender_age_label_df.csv',
                                 img_dir='dataset/Image',
                                 transform=transform)

# DataLoader 생성
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)


In [24]:
project_id = "Project ID"

# 인증 설정
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)

# 생성할 태그 정의
gender_tags = ['male', 'female']
age_class_tags = ['old', 'adult', 'young']

# 기존 태그 확인
existing_tags = trainer.get_tags(project_id)
existing_tag_dict = {tag.name: tag.id for tag in existing_tags}

# 태그 생성 함수
def get_or_create_tag(trainer, project_id, tag_name, existing_tags):
    if tag_name in existing_tags:
        return existing_tags[tag_name]
    else:
        new_tag = trainer.create_tag(project_id, tag_name)
        return new_tag.id

# gender와 age_class 태그 생성
gender_tag_ids = {}
age_class_tag_ids = {}

# gender 태그 생성
for gender in gender_tags:
    gender_tag_ids[gender] = get_or_create_tag(trainer, project_id, gender, existing_tag_dict)

# age_class 태그 생성
for age_class in age_class_tags:
    age_class_tag_ids[age_class] = get_or_create_tag(trainer, project_id, age_class, existing_tag_dict)


In [28]:
# Azure Custom Vision 설정
project_id = "Project ID"

# 인증 설정
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)

# CSV 파일 읽기
csv_path = 'dataset/gender_age_label_df.csv'
dataset_df = pd.read_csv(csv_path)

# 기존 태그 가져오기
existing_tags = trainer.get_tags(project_id)
existing_tag_dict = {tag.name: tag.id for tag in existing_tags}

# 이미지 리스트 생성 및 태그 자동 레이블링
image_list = []
for _, row in dataset_df.iterrows():
    img_filename = row['img_filename']
    image_path = f"dataset/Image/{img_filename}"
    
    # 해당 이미지의 태그 ID 리스트 생성
    tag_ids = [
        existing_tag_dict[row['gender']],      # gender 태그
        existing_tag_dict[row['age_class']]    # age_class 태그
    ]
    
    try:
        with open(image_path, "rb") as image_contents:
            image_list.append(
                ImageFileCreateEntry(
                    name=img_filename,
                    contents=image_contents.read(),
                    tag_ids=tag_ids
                )
            )
    except FileNotFoundError:
        print(f"이미지를 찾을 수 없습니다: {image_path}")

# 배치 업로드 함수
def upload_images(trainer, project_id, image_batch):
    try:
        upload_result = trainer.create_images_from_files(
            project_id,
            ImageFileCreateBatch(images=image_batch)
        )
        
        if not upload_result.is_batch_successful:
            print("이미지 배치 업로드에 실패했습니다.")
            for image in upload_result.images:
                print(f"이미지: {image.source_url}, 상태: {image.status}")
                if image.status == "OKDuplicate":
                    print("중복 이미지입니다. 이미 프로젝트에 존재합니다.")
                elif image.status != "OK":
                    print(f"업로드 실패 이유: {image.status}")
        else:
            print(f"{len(image_batch)}개 이미지가 성공적으로 업로드되었습니다.")
    except Exception as e:
        print(f"업로드 중 오류 발생: {str(e)}")

# 배치 크기 설정 (Azure의 제한으로 인해 64개씩 처리)
BATCH_SIZE = 64

# 배치 단위로 이미지 업로드
for i in range(0, len(image_list), BATCH_SIZE):
    batch = image_list[i:i + BATCH_SIZE]
    upload_images(trainer, project_id, batch)


이미지 배치 업로드에 실패했습니다.
이미지: IN_H00276_SN1_090405_15487.png, 상태: OKDuplicate
중복 이미지입니다. 이미 프로젝트에 존재합니다.
이미지: IN_H00276_SN1_090405_15439.png, 상태: OKDuplicate
중복 이미지입니다. 이미 프로젝트에 존재합니다.
이미지: IN_H00276_SN1_090405_15457.png, 상태: OKDuplicate
중복 이미지입니다. 이미 프로젝트에 존재합니다.
이미지: IN_H00276_SN1_090405_15484.png, 상태: OKDuplicate
중복 이미지입니다. 이미 프로젝트에 존재합니다.
이미지: IN_H00276_SN1_090405_15478.png, 상태: OKDuplicate
중복 이미지입니다. 이미 프로젝트에 존재합니다.
이미지: IN_H00276_SN1_090405_15451.png, 상태: OKDuplicate
중복 이미지입니다. 이미 프로젝트에 존재합니다.
이미지: IN_H00276_SN1_090405_15436.png, 상태: OKDuplicate
중복 이미지입니다. 이미 프로젝트에 존재합니다.
이미지: IN_H00276_SN1_090405_15430.png, 상태: OKDuplicate
중복 이미지입니다. 이미 프로젝트에 존재합니다.
이미지: IN_H00276_SN1_090408_17306.png, 상태: OKDuplicate
중복 이미지입니다. 이미 프로젝트에 존재합니다.
이미지: IN_H00276_SN1_090405_15472.png, 상태: OKDuplicate
중복 이미지입니다. 이미 프로젝트에 존재합니다.
이미지: IN_H00276_SN1_090408_17303.png, 상태: OKDuplicate
중복 이미지입니다. 이미 프로젝트에 존재합니다.
이미지: IN_H00276_SN1_090405_15427.png, 상태: OKDuplicate
중복 이미지입니다. 이미 프로젝트에 존재합니다.
이미지: IN_H00276_SN1_0